In [48]:
!pip install fbprophet

     |████████████████████████████████| 61kB 949kB/s eta 0:00:01
  ERROR: Complete output from command /Library/Frameworks/Python.framework/Versions/3.6/bin/python3.6 -u -c 'import setuptools, tokenize;__file__='"'"'/private/var/folders/7b/8c7scrn952d29tp2ft1gnxdh0000gn/T/pip-install-yv860pry/fbprophet/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/7b/8c7scrn952d29tp2ft1gnxdh0000gn/T/pip-wheel-tuulx3sa --python-tag cp36:
  ERROR: running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib
  creating build/lib/fbprophet
  creating build/lib/fbprophet/stan_model
  INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_888a84912910fa0a45b9d614b75bb8a8 NOW.
  error: command '/Users/parthjsoni/anaconda2/bin/clang' failed with exit status 1
  ----------------------------------------
  ERROR: Faile

In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go 
import seaborn as sns
import plotly
import plotly.express as px
# from fbprophet.plot import plot_plotly
from fbprophet import Prophet
# from fbprophet.plot import add_changepoints_to_plot
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, plot_mpl
import plotly.offline as py
init_notebook_mode(connected=True)

In [3]:
cnf = '#393e46' # confirmed - grey
dth = '#ff2e63' # death - red
rec = '#21bf73' # recovered - cyan
act = '#fe9801' # active case - yellow

In [4]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [5]:
import warnings
warnings.filterwarnings('ignore')
import os

In [6]:
from opencage.geocoder import OpenCageGeocode
from pprint import pprint

In [7]:
key='807b3c01fb4b4dea97b8bd9d86b2e83a'
geocoder=OpenCageGeocode(key)

In [8]:
ls

COVID19_line_list_data.csv*         time_series_covid_19_confirmed.csv*
COVID19_open_line_list.csv*         time_series_covid_19_deaths.csv*
Corona Virus.ipynb                  time_series_covid_19_recovered.csv*
covid_19_data.csv*


In [9]:
df1=pd.read_csv('covid_19_data.csv',parse_dates=['ObservationDate'])

In [10]:
list(df1)

['SNo',
 'ObservationDate',
 'Province/State',
 'Country/Region',
 'Last Update',
 'Confirmed',
 'Deaths',
 'Recovered']

In [11]:
# list_lat=[]

# list_long=[]

# for x in range(len(df1)):
#     state=row['Province/State']
#     country=row['Country/Region']
#     query=str(state)+','+str(country)
#     result=geocoder.geocode(query)
    
#     lat = result[0]['geometry']['lat']
#     long = result[0]['geometry']['lng']
    
#     list_lat.append(lat)
#     list_long.append(long)
    
# df1['latitide']=list_lat
# df1['longitude']=list_long

In [13]:
df1["Active"]=df1['Confirmed']-df1['Deaths']-df1['Recovered']

df1['Country/Region']=df1['Country/Region'].replace('Mainland China', 'China')

df1['Province/State']=df1['Province/State'].fillna('')

# case=case.fillna(0)
df1['Confirmed']=df1['Confirmed'].astype(int)
df1['Deaths']=df1['Deaths'].astype(int)
df1['Recovered']=df1['Recovered'].astype(int)
df1['Active']=df1['Active'].astype(int)

In [14]:
df1.head(20).style.background_gradient(cmap='PRGn')

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Active
0,1,2020-01-22 00:00:00,Anhui,China,1/22/2020 17:00,1,0,0,1
1,2,2020-01-22 00:00:00,Beijing,China,1/22/2020 17:00,14,0,0,14
2,3,2020-01-22 00:00:00,Chongqing,China,1/22/2020 17:00,6,0,0,6
3,4,2020-01-22 00:00:00,Fujian,China,1/22/2020 17:00,1,0,0,1
4,5,2020-01-22 00:00:00,Gansu,China,1/22/2020 17:00,0,0,0,0
5,6,2020-01-22 00:00:00,Guangdong,China,1/22/2020 17:00,26,0,0,26
6,7,2020-01-22 00:00:00,Guangxi,China,1/22/2020 17:00,2,0,0,2
7,8,2020-01-22 00:00:00,Guizhou,China,1/22/2020 17:00,1,0,0,1
8,9,2020-01-22 00:00:00,Hainan,China,1/22/2020 17:00,4,0,0,4
9,10,2020-01-22 00:00:00,Hebei,China,1/22/2020 17:00,1,0,0,1


In [15]:
df1=df1.drop(['SNo', 'Last Update'], axis=1)
df1=df1.rename(columns={'ObservationDate': 'Date', 'Province/State': 'State', 'Country/Region':'Country'})
df1.isnull().sum()


Date         0
State        0
Country      0
Confirmed    0
Deaths       0
Recovered    0
Active       0
dtype: int64

In [16]:
temp=df1.groupby('Date')[('Confirmed','Deaths','Recovered','Active')].sum().reset_index()

In [17]:
temp=temp.sort_values('Date', ascending=False)
temp['Confirmed']=temp['Confirmed'].astype(int)
temp['Deaths']=temp['Deaths'].astype(int)
temp['Recovered']=temp['Recovered'].astype(int)
temp.head().style.background_gradient(cmap='PRGn')


,Date,Confirmed,Deaths,Recovered,Active
48,2020-03-10 00:00:00,118582,4262,64404,49916
47,2020-03-09 00:00:00,113582,3996,62512,47074
46,2020-03-08 00:00:00,109835,3803,60695,45337
45,2020-03-07 00:00:00,105836,3558,58359,43919
44,2020-03-06 00:00:00,101800,3460,55866,42474


In [18]:
#Confirmed ALL
fig = go.Figure()
fig.update_layout(template='plotly_dark')
fig.add_trace(go.Scatter(x=temp['Date'], 
                         y=temp['Confirmed'],
                         mode='lines+markers',
                         name='Confirmed',
                         line=dict(color='Yellow', width=2)))
fig.add_trace(go.Scatter(x=temp['Date'], 
                         y=temp['Deaths'],
                         mode='lines+markers',
                         name='Deaths',
                         line=dict(color='Red', width=2)))
fig.add_trace(go.Scatter(x=temp['Date'], 
                         y=temp['Recovered'],
                         mode='lines+markers',
                         name='Recovered',
                         line=dict(color='Green', width=2)))
fig.add_trace(go.Scatter(x=temp['Date'], 
                         y=temp['Active'],
                         mode='lines+markers',
                         name='Active',
                         line=dict(color='Blue', width=2)))

fig.show()


In [19]:
# Visualising Confirm and Active case only

china_vs_rest=df1.copy()
china_vs_rest=china_vs_rest[china_vs_rest['Country']!='China']
china_vs_rest=china_vs_rest.groupby(['Date','Country'])['Confirmed','Active'].agg(sum)
china_vs_rest.columns=['Confirmed All', 'Active All']
china_vs_rest=china_vs_rest.reset_index()
#Confirmed Cases
fig1 = px.line(china_vs_rest, x="Date", y="Confirmed All", color="Country",
              line_group="Country", hover_name="Country")
fig1.update_layout(template='plotly_dark')
#Active Cases
fig2 = px.line(china_vs_rest, x="Date", y="Active All", color="Country",
              line_group="Country", hover_name="Country")
fig2.update_layout(template='plotly_dark')
 

In [20]:
fig1

In [21]:
grouped_multiple=df1.groupby('Date').agg({'Deaths':'sum', 'Recovered':'sum','Active':'sum','Confirmed':'sum'})
grouped_multiple.columns=['Death All', 'Recovered All', 'Active All', 'Confirmed All']
grouped_mulitple=grouped_multiple.reset_index()
grouped_multiple['Difference_world']=grouped_multiple['Confirmed All'].diff().shift(-1)

grouped_multiple['Death_All%']=grouped_multiple.apply(lambda row: ((row['Death All'])/(row['Confirmed All']))*100, axis=1)
grouped_multiple['Active_All%']=grouped_multiple.apply(lambda row: ((row['Active All'])/(row['Confirmed All']))*100, axis=1)
grouped_multiple['Recovered_All%']=grouped_multiple.apply(lambda row: ((row['Recovered All'])/(row['Confirmed All']))*100, axis=1)
grouped_multiple['World_Growth_Rate']=grouped_multiple.apply(lambda row: ((row['Difference_world'])/(row['Confirmed All']))*100, axis=1)

In [22]:
grouped_multiple=grouped_multiple.reset_index()

In [23]:
fig=go.Figure()

fig.add_trace(go.Scatter(x=grouped_multiple['Date'], 
                         y=grouped_multiple['Death_All%'],
                         mode='lines+markers',
                         name='Death Rate',
                         line=dict(color='red', width=2)))
fig.add_trace(go.Scatter(x=grouped_multiple['Date'], 
                         y=grouped_multiple['Recovered_All%'],
                         mode='lines+markers',
                         name='Recovered Rate',
                         line=dict(color='Green', width=2)))
fig.add_trace(go.Scatter(x=grouped_multiple['Date'], 
                         y=grouped_multiple['World_Growth_Rate'],
                         mode='lines+markers',
                         name='Growth Rate Confirmed',
                         line=dict(color='Yellow', width=2)))
fig.add_trace(go.Scatter(x=grouped_multiple['Date'], 
                         y=grouped_multiple['Active_All%'],
                         mode='lines+markers',
                         name='Active Rate',
                         line=dict(color='Blue', width=2)))
fig.update_layout(template='plotly_dark')
fig.show()
grouped_multiple.tail()

,Date,Death All,Recovered All,Active All,Confirmed All,Difference_world,Death_All%,Active_All%,Recovered_All%,World_Growth_Rate
44,2020-03-06,3460,55866,42474,101800,4036.0,3.398821,41.722986,54.878193,3.964637
45,2020-03-07,3558,58359,43919,105836,3999.0,3.361805,41.497222,55.140973,3.778487
46,2020-03-08,3803,60695,45337,109835,3747.0,3.462466,41.277371,55.260163,3.411481
47,2020-03-09,3996,62512,47074,113582,5000.0,3.518163,41.444947,55.036890,4.402106
48,2020-03-10,4262,64404,49916,118582,NaN,3.594137,42.094078,54.311784,NaN


In [24]:
# Checking if the growth rate is accelerating or decellerating
mortality=df1.copy()

mortality = mortality.groupby(['Date','Country']).agg({'Confirmed':['sum'], 'Recovered':['sum'],'Deaths':['sum'],'Active':['sum']})
mortality.columns=['Confirmed All', 'Recovered All', 'Death All','Active All']
mortality=mortality.reset_index()
mortality=mortality[mortality['Death All']!=0]
mortality=mortality[mortality['Confirmed All']!=0]
mortality=mortality[mortality['Active All']!=0]
mortality['mortality rate']=mortality.apply(lambda row:((row['Death All']+1)/(row['Confirmed All']+1))*100,axis=1)
#filtering out where mortality rate is more than 10
d= mortality[mortality['mortality rate'] <10]

# Filtering the data to take into consideration those contry with more than 100 confirmed cases
dd=d[d['Confirmed All']>100]
fig=px.line(dd, x='Date', y='mortality rate', color='Country',line_group='Country',hover_name='Country')
fig.update_layout(template='plotly_dark')
fig.show(0)

In [25]:

# Checking if the Active Rate is accelerating or decellerating
act_1=df1.copy()

act_1 = act_1.groupby(['Date','Country']).agg({'Confirmed':['sum'], 'Recovered':['sum'],'Deaths':['sum'],'Active':['sum']})
act_1.columns=['Confirmed All', 'Recovered All', 'Death All','Active All']
act_1=act_1.reset_index()
act_1=act_1[act_1['Death All']!=0]
act_1=act_1[act_1['Confirmed All']!=0]
act_1=act_1[act_1['Active All']!=0]
act_1['active rate']=act_1.apply(lambda row:((row['Active All']+1)/(row['Confirmed All']+1))*100,axis=1)
#filtering out where mortality rate is more than 10
a= act_1[act_1['active rate'] >10]
# print(a.head(10))
# Filtering the data to take into consideration those contry with more than 100 confirmed cases
aa=a[a['Confirmed All']>100]
fig=px.line(aa, x='Date', y='active rate', color='Country',line_group='Country',hover_name='Country')
fig.update_layout(template='plotly_dark')
fig.show(0)

In [46]:
from fbprophet.plot import plot_plotly

ModuleNotFoundError: No module named 'fbprophet.plot'

In [47]:
from fbprophet.plot import plot_plotly
from fbprophet.plot import add_changepoints_to_plot

ModuleNotFoundError: No module named 'fbprophet.plot'

In [31]:
#Modelling Active Cases 
floorVar=0.8
worldPop=25000

#Modelled total Confirmed Cases
active_training_dataset=pd.DataFrame(df1.groupby('Date')['Active'].agg('sum').reset_index()).rename(columns={'Date':'ds','Active':'y'})
active_training_dataset['floor']=active_training_dataset.y*floorVar
active_training_dataset['cap']=active_training_dataset.y+worldPop

# Modelling Mortality Rate
mortality_training_dataset=pd.DataFrame(mortality.groupby('Date')['mortality rate'].agg('mean').reset_index()).rename(columns={'Date':'ds','mortality rate':'y'})

# Modelling Death
death_training_dataset=pd.DataFrame(df1.groupby('Date')['Deaths'].agg('sum').reset_index()).rename(columns={'Date':'ds','Deaths':'y'})

In [55]:
m=Prophet(growth='linear',interval_width=0.9,
    yearly_seasonality=False,
    daily_seasonality=True,
    seasonality_mode = 'additive'
    )

m.fit(active_training_dataset)
future=m.make_future_dataframe(period=61)
future['cap']=active_training_dataset.y+worldPop
future['floor']=active_training_dataset.y*floorVar
active_forcast=m.predict(future)

#Mortality rate
m_mortality= Prophet()
m_mortality.fit(mortality_training_dataset)
mortality_future=m_mortality,make_future_dataframe(period=31)
mortality_forecastm_mortality.predict(mortality_future)

#Death Rate 
m2=Prophet(interval_width=0.9)
m2.fit(death_training_dataset)
future2=m2.make_future_dataframe(periods=7)
death_forecast=m2.predict(future2)

TypeError: __init__() got an unexpected keyword argument 'seasonality_mode'

In [50]:
import fbprophet
print(fbprophet.__version__)

0.2.1
